In [3]:
############# MNIST 데이터 로딩 및 신경망 정의##########################

# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch.quantization
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

# Training settings
batch_size = 64

# MNIST Dataset
# MNIST 데이터 셋을 가져와 train, test 구분
train_dataset = datasets.MNIST(root="datasets/",
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)
test_dataset = datasets.MNIST(root="datasets/",
                                train=False,
                                transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
# 각 데이터 셋에서 용도에 맞게 데이터를 랜덤하게 섞어서 Batch size 수에 맞게 가져온다
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
                                            shuffle=False)

# 신경망 구조 정의
class Net(nn.Module):
    
    # 초기화
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1_out_np = np.zeros((1, 3, 24, 24))
        self.mp1_out_np = np.zeros((1, 3, 12, 12))
        self.conv2_out_np = np.zeros((3, 3, 8, 8))
        #self.conv2_out_np = np.zeros((3, 3, 8, 8))
        self.conv3_out_np = np.zeros((1, 3, 8, 8))
        #self.conv3_out_np = np.zeros((1, 3, 8, 8))
        self.mp2_out_np = np.zeros((1, 3, 4, 4))
        #self.mp2_out_np = np.zeros((1, 3, 4, 4))
        self.fc_in_np = np.zeros((1, 49))
        #self.fc_in_np = np.zeros((1, 48))
        self.fc_out_np = np.zeros((1, 10))
        
        # 커널 크기가 5, 입력 채널이 1, 출력 채널이 3인 CNN 층 구성
        # 출력 채널은 커널의 개수를 의미
        self.conv1 = nn.Conv2d(1, 3, kernel_size=5, bias=False) # Num of weight = 
        
        # 커널 크기가 5, 입력 채널이 3, 출력 채널이 3인 CNN 층 구성
        self.conv2 = nn.Conv2d(3, 3, kernel_size=5, bias=False)
        
        #self.conv2 = nn.Conv2d(3, 3*6, kernel_size=5, groups=3, bias=False)
        
        #self.conv3 = nn.Conv2d(3*6, 6, kernel_size=1)
        
        # Max Pooling Layer, 파라미터는 kernel size 의미, 4개의 값 중 최댓값을 출력한다.
        self.mp = nn.MaxPool2d(2)
        
        # Fully Connected Layer
        self.fc_1 = nn.Linear(48, 10) # Num of Weight = 480
        #self.fc_1 = nn.Linear(96, 10) # Num of Weight = 480
        
    def forward(self, x):
        # 텐서를 일자로 펴기 위해 x의 한 원소의 크기 계산
        in_size = x.size(0)
        
        # CNN 층 출력이 최대 풀링 층을 지나 활성함수 ReLU를 지난다.
        x = self.conv1(x)
        self.conv1_out_np = x.detach().numpy()
        
        x = F.relu(self.mp(x))
        self.mp1_out_np = x.detach().numpy()

        x = self.conv2(x)
        self.conv2_out_np = x.detach().numpy()
        
        x = self.conv3(x)
        self.conv3_out_np = x.detach().numpy()
        
        x = F.relu(self.mp(x))
        self.mp2_out_np = x.detach().numpy()
        
        # 텐서를 Fully Connected Layer에 넣기 위해 일자로 편다
        x = x.view(in_size, -1) # flatten the tensor
        self.fc_in_np = x.detach().numpy()
        
        # 일자로 편 텐서로 Fully Connected Layer 계산
        x = self.fc_1(x)
        self.fc_out_np = x.detach().numpy()
        
        # 출력층의 활성함수는 Softmax 사용
        return F.log_softmax(x)
    
# Instantiation    
model = Net()

# 최적화 방법으로 Momentum(= 0.5) 방식을 추가한 Stochastical Gradient Descent를 사용한다.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

################### 신경망 학습 및 실행 ##################################

def train(epoch):
    # 해당 모델을 training 상태로 설정하는 함수
    model.train()
    
    # train_loader로 batch_idx, data, target 각각 반복마다 
    # train_loader의 인덱스를 하나씩 늘려가며 data를 넣는다.
    for batch_idx, (data, target) in enumerate(train_loader):
        
        #print(batch_idx)
        
        # Pytorch에서 다룰 수 있게 Variable로 변경
        data, target = Variable(data), Variable(target)
        
        #print(np.shape(data))
        
        # 다음 최적화를 위해 전에 저장되어있던 값을 초기화
        optimizer.zero_grad()
        
        # model을 feed-forward 되어 나온 출력값
        output = model(data)
        
        # Negative Log Likelihood 로 Loss를 계산 
        loss = F.nll_loss(output, target)
        
        # Gradient 값을 계산한다.
        loss.backward()
        
        # back propagation을 통한 가중치값 최적화를 한번 진행
        optimizer.step()
        
        # batch_idx가 10의 배수일 때 => 10번마다
        if batch_idx % 10 == 0:
            #각 값을 모니터 출력
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                # Percentage로 나타내기 위해 100을 곱함
                100. * batch_idx / len(train_loader), loss.item()))
                # data[0] is deprecated

# 테스트에 사용할 함수 정의
def test():
    # 해당 모델을 evaluation 상태로 설정하는 함수
    model.eval()
    # loss 값 stacking 위해 초기화
    test_loss = 0
    # 맞춘 수를 stacking 위해 초기화
    correct = 0
    
    
    for data, target in test_loader:
        
        # 각 배열을 PyTorch로 다룰 수 있는 Variable로 만들어준다
        # volatile was removed and now has no effect
        data, target = Variable(data, volatile=True), Variable(target)
        
        # model을 feed-forward 되어 나온 출력값
        output = model(data)
        
        # sum up batch loss
        #loss 값 stacking
        test_loss += F.nll_loss(output, target, size_average=False).item()
        
        # get the index of the max log-probability
        # max() : 1과 출력 data 중 최대값 출력
        pred = output.data.max(1, keepdim=True)[1]
        
        # view_as(pred) == view(pred.size())
        # => 같은 데이터를 갖는 텐서 배열을 1*n 의 새로운 배열의 텐서로 반환한다.
        # cpu() : CUDA 처리가 불가능하여 CPU로 처리하게 하는 함수
        # sum() : 텐서의 모든 원소 합 반환
        # eq() : 입력받은 두 배열을 비교하여 대응되는 원소값이 갇으면 1,
        # 다르면 0을 갖는 같은 크기의 배열을 반환한다
        # pred와 target 비교하여 둘이 같으면 1을 갖는 배열의 합, 즉 pred 와 target이 맞는 수를 센다 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    # 데이터 셋의 크기로 Loss를 나눠준다    
    test_loss /= len(test_loader.dataset)
    
    # 각 결과값을 모니터 출력
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 10):
    train(epoch)
    test()

C:\Users\user\AppData\Local\Temp\ipykernel_33968\2241577816.py:106: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299814
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.298216
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.318568
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.297652
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.292070
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.279231
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.259009
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.256701
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.266300
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.255168
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.251190
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.262122
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.238704
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.213689
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.241606
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.194288
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.179919
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.104992
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.023214
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.004714
Train Epoch: 1 [12800/60000 (

C:\Users\user\AppData\Local\Temp\ipykernel_33968\2241577816.py:169: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
c:\Users\user\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.3013, Accuracy: 9047/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.240111
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.161777
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.340523
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.389488
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.099673
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.257463
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.229250
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.308691
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.337729
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.585684
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.322422
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.243561
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.168047
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.207023
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.204207
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.267316
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.361105
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.213788
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.322149
Train Epoch: 2 [12

In [32]:
################# 샘플 통과 후 결과값 추출 #################
from PIL import Image

# 샘플을 이용한 테스트
def sample_test():
    # 해당 모델을 evaluation 상태로 설정하는 함수
    model.eval()
    # loss 값 stacking 위해 초기화
    test_loss = 0
    # 맞춘 수를 stacking 위해 초기화
    correct = 0
    
    ############## 테스트 샘플 (0 한장) 불러오기 ####################
    
    # target 생성 (label : 0)
    target = Variable(torch.,6]))
    
    # data 생성 (0_0.bmp)

    ######################################
   
    
    img = Image.open("C:\\Users\\user\\Desktop\\mnist\\archive\\testSample\\testSample\\img_205.jpg", "r")
    np_img = np.array(img)
    np_img_re = np.reshape(np_img, (1,1,28,28))
    #print(np_img_re)
    #print(Variable(torch.tensor((np_img_re), dtype = torch.float32)))
    #np.savetxt('image_test4_num2.mem', np_img_re, fmt='%1.2x',delimiter = " ")
    

  
     
    
    
    # 0 - 255 => 0 - 1 로 정규화, np.array => tensor 변환
    data = Variable(torch.tensor(((np_img_re / 255)), dtype = torch.float32))
    float_my = [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0196, 0.0000, 0.0000, 0.0118, 0.0000, 0.0000, 0.0000, 0.0235,
           0.0000, 0.0000, 0.0196, 0.0275, 0.0039, 0.0000, 0.0000, 0.0078,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0235, 0.0627, 0.0314, 0.0000,
           0.0431, 0.0235, 0.0000, 0.0000, 0.0000, 0.0510, 0.0314, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0549, 0.0078, 0.0000, 0.0118, 0.0000, 0.0000, 0.0000, 0.0510,
           0.0000, 0.0000, 0.0039, 0.0275, 0.0157, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0039, 0.0078, 0.0000, 0.0118, 0.0588, 0.0235, 0.0000,
           0.0235, 0.0039, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0118,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0157, 0.0157, 0.0118, 0.0000, 0.0000, 0.0000, 0.0000, 0.0118,
           0.0000, 0.0157, 0.0510, 0.0078, 0.0000, 0.0235, 0.0196, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0078, 0.0039, 0.0078, 0.0196, 0.0353, 0.0353, 0.0118, 0.0000,
           0.1020, 0.0078, 0.0000, 0.0000, 0.0157, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0078, 0.0000, 0.0314, 0.4471, 0.9490, 1.0000, 0.9216,
           0.7529, 0.4157, 0.0824, 0.0000, 0.0000, 0.0118, 0.0118, 0.0314,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.2784, 0.7490, 1.0000, 0.9725, 0.9490, 1.0000,
           0.9882, 1.0000, 0.8471, 0.3451, 0.0000, 0.0000, 0.0196, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0078, 0.0000, 0.0549, 0.0000, 0.0000, 0.0039,
           0.0471, 0.4627, 0.9922, 0.9333, 1.0000, 0.7686, 0.4471, 0.4353,
           0.7255, 0.9490, 1.0000, 0.9765, 0.2863, 0.0000, 0.0000, 0.0157,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0196, 0.0000, 0.0000, 0.0314, 0.0196, 0.0000, 0.0235,
           0.3176, 0.9765, 0.9882, 0.9686, 0.2941, 0.0471, 0.0353, 0.0000,
           0.0392, 0.0549, 0.6275, 0.9608, 0.8314, 0.0902, 0.0471, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0118, 0.0000, 0.0000, 0.0118, 0.0196, 0.0000, 0.2353,
           1.0000, 0.9647, 0.8863, 0.2039, 0.0353, 0.0000, 0.0078, 0.0824,
           0.0000, 0.0000, 0.0039, 0.8471, 0.9647, 0.2078, 0.0157, 0.0314,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0078, 0.0000, 0.0000, 0.0353, 0.0039, 0.0000, 0.0235, 0.6314,
           0.9843, 0.8314, 0.0784, 0.0824, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0157, 0.0588, 0.1176, 0.8863, 1.0000, 0.2980, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0078, 0.0392, 0.0000, 0.0000, 0.1882, 0.9451,
           0.9216, 0.1490, 0.0118, 0.0000, 0.0118, 0.0471, 0.0157, 0.0157,
           0.0118, 0.0000, 0.7647, 0.9098, 1.0000, 0.1608, 0.0824, 0.0353,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0078, 0.0000, 0.0000, 0.0627, 0.3804, 1.0000,
           0.5255, 0.0000, 0.0471, 0.0000, 0.0039, 0.0157, 0.0000, 0.0118,
           0.1608, 0.6431, 0.9961, 1.0000, 0.9882, 0.0784, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0078, 0.0000, 0.0000, 0.0392, 0.4588, 0.9961,
           0.1373, 0.0157, 0.0000, 0.0000, 0.0000, 0.1176, 0.3216, 0.6000,
           0.9412, 1.0000, 0.9098, 0.9765, 0.1765, 0.0000, 0.0941, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0078, 0.0196, 0.0039, 0.0000, 0.4353, 0.9725,
           0.6941, 0.4039, 0.5059, 0.6980, 0.6941, 0.9412, 1.0000, 0.9686,
           1.0000, 0.9608, 1.0000, 0.4824, 0.0000, 0.0235, 0.0000, 0.0392,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0078, 0.0000, 0.0039, 0.0196, 0.2667, 0.9608,
           0.9882, 1.0000, 0.9804, 1.0000, 1.0000, 0.7294, 0.8627, 0.9569,
           0.9765, 1.0000, 0.5059, 0.0000, 0.0039, 0.0627, 0.0000, 0.0078,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0353, 0.0000, 0.0431, 0.0039, 0.0000, 0.0000, 0.0588, 0.5412,
           0.9647, 1.0000, 0.7647, 0.8471, 0.5333, 0.0588, 0.0824, 1.0000,
           1.0000, 0.9137, 0.1255, 0.0510, 0.0000, 0.0000, 0.0824, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0118, 0.0078, 0.0118, 0.0078, 0.0196, 0.2510,
           0.5451, 0.3020, 0.0157, 0.0471, 0.0706, 0.0000, 0.4235, 1.0000,
           1.0000, 0.4549, 0.0000, 0.0235, 0.0000, 0.0000, 0.0314, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0235, 0.0000, 0.0275, 0.0000, 0.0000, 0.0157, 0.0000, 0.0392,
           0.0000, 0.0000, 0.0431, 0.0000, 0.0000, 0.1255, 1.0000, 0.9922,
           0.7608, 0.0078, 0.0549, 0.0000, 0.0275, 0.0353, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0392, 0.0000, 0.0235, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0706, 0.0157, 0.0196, 0.0784, 0.6549, 1.0000, 0.9255,
           0.3137, 0.0000, 0.0039, 0.0000, 0.0588, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0196, 0.0353, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0863, 0.9412, 0.9765, 0.8784,
           0.0157, 0.0353, 0.0000, 0.0157, 0.0000, 0.0000, 0.0039, 0.0353,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0118, 0.0157, 0.0039, 0.0196, 0.0157, 0.0000, 0.0000, 0.0000,
           0.0275, 0.0000, 0.0784, 0.0000, 0.2941, 1.0000, 0.9725, 0.3020,
           0.0000, 0.0000, 0.0039, 0.0157, 0.0000, 0.0471, 0.0000, 0.0392,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0196, 0.0000, 0.0000, 0.0157,
           0.0000, 0.0235, 0.0000, 0.0039, 0.4392, 0.9922, 0.9961, 0.0118,
           0.0196, 0.0000, 0.0000, 0.0157, 0.0078, 0.0039, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0157, 0.0196, 1.0000, 0.9686, 0.6000, 0.0235,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0980, 0.0000, 0.0000, 0.1020, 0.6314, 1.0000, 0.1216, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0745, 0.0000, 0.0275, 0.0118, 0.0157, 0.0000, 0.0353,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0510, 0.0196, 0.0196, 0.0000, 0.0353, 0.0000, 0.0078, 0.0235,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000]]
    
    print(data)
    from fxpmath import Fxp
    #b = Fxp(1.1445, signed=True, n_word=8, n_frac=4)
    #print(b.hex())

    for i in range(28):
        for j in range(28):
        
            x = Fxp(float_my[i][j], signed=True, n_word=8, n_frac=4)
            #print(x)         # float value
            #print(x.bin())   # binary representation
            print(x.hex())   # hex repr
            #print(x.val)     # raw val (decimal of binary stored)
            
    
    # model을 feed-forward 되어 나온 출력값
    output = model(data)
    #print(np.shape(output))
    
    # sum up batch loss
    # loss 값 stacking
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    
    
    # get the index of the max log-probability
    # max() : 1과 출력 data 중 최대값 출력
    pred = output.data.max(1, keepdim=True)[1]
    
    #view_as(pred) == view(pred.size())
    # => 같은 데이터를 갖는 텐서 배열을 1*n 의 새로운 배열의 텐서로 반환한다.
    # cpu() : CUDA 처리가 불가능하여 CPU로 처리하게 하는 함수
    
    # sum() : 텐서의 모든 원소 합 반환
    # eq() : 입력받은 두 배열을 비교하여 대응되는 원소값이 갇으면 1,
    # 다르면 0을 갖는 같은 크기의 배열을 반환한다
    # pred와 target 비교하여 둘이 같으면 1을 갖는 배열의 합, 즉 pred 와 target이 맞는 수를 센다 
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    print(correct)
    
    # 데이터 셋의 크기로 Loss를 나눠준다    
    #test_loss /= len(test_loader.dataset)
    
    # 각 결과값을 모니터 출력
    #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        #test_loss, correct, len(test_loader.dataset),
        #100. * correct / len(test_loader.dataset)))
    

sample_test()

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0196, 0.0000, 0.0000, 0.0118, 0.0000, 0.0000, 0.0000, 0.0235,
           0.0000, 0.0000, 0.0196, 0.0275, 0.0039, 0.0000, 0.0000, 0.0078,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0235, 0.0627, 0.0314, 0.0000,
           0.0431, 0.0235, 0.0000, 0.0000, 0.0000, 0.0510, 0.0314, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0549, 0.0078, 0.0000, 0.0118, 0.0000, 0.0000, 0.0000, 0.0510,
           0.0000, 0.0000, 0.0039, 0.0275, 0.0157, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0039, 0.0078, 0.0000, 0.0118, 0.0588, 0.0235, 0.0000,
           0.0235, 0.0039, 0.0000, 0.0000, 

C:\Users\user\AppData\Local\Temp\ipykernel_35728\2533204397.py:106: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [5]:
############## Conv1 가중치 값 HEX 추출 ############

# float => int
conv1_weight =  [[-0.0773,  0.2015,  0.1873, -0.0046,  0.1555],
        [-0.0696, -0.2170, -0.0775,  0.1051,  0.2148],
        [ 0.0901, -0.1994, -0.0626,  0.1994,  0.3410],
        [-0.0999, -0.0086, -0.0725,  0.1980,  0.2159],
        [ 0.0177, -0.1017, -0.1570, -0.0096,  0.2055],
[-0.0953,  0.0423,  0.3730,  0.2413,  0.0263],
        [-0.1221,  0.1831,  0.2278,  0.3930,  0.2553],
        [ 0.1548,  0.3197,  0.5521,  0.8877,  0.4599],
        [ 0.0680,  0.5098,  0.8524,  0.5987,  0.1161],
        [ 0.1649,  0.1793,  0.4654,  0.3942,  0.2900],
[ 0.2414,  0.0408, -0.2019,  0.0548, -0.1531],
        [ 0.1850,  0.3362,  0.5191,  0.5919,  0.4848],
        [-0.0308,  0.4638,  0.5315,  0.3755,  0.2551],
        [-0.1318, -0.3182, -0.1797, -0.0086,  0.0633],
        [-0.2279, -0.5504, -0.6544, -0.3949, -0.0072]]


float_conv1_weight_1 =  torch.tensor((model.conv1.weight.data[0][0]), dtype = torch.float32)
float_conv1_weight_2 =  torch.tensor((model.conv1.weight.data[1][0]), dtype = torch.float32)
float_conv1_weight_3 =  torch.tensor((model.conv1.weight.data[2][0]), dtype = torch.float32)

arrary_conv1_weight = np.zeros((15,5))

#print("Signed")
print(float_conv1_weight_1)
print(float_conv1_weight_2)
print(float_conv1_weight_3)

from fxpmath import Fxp
   
print("weights")
for i in range(15):
    for j in range(5):
        
        x = Fxp(conv1_weight[i][j], signed=True, n_word=8, n_frac=4)
        #print(x)
        arrary_conv1_weight[i][j] = x.bin(); 
        # # float value
        #print(x.bin())   # binary representation
        print(x.hex())   # hex repr
        #print(x.val)     # raw val (decimal of binary stored)
#print(int_conv1_weight_2)
#print(int_conv1_weight_3)
#print(int_conv1_weight_1)
#print(arrary_conv1_weight)

# signed int => unsigned int
#for i in range(5):
    #for j in range(5):
       
        #if int_conv1_weight_1[i][j] < 0:
        #    int_conv1_weight_1[i][j] += 256
        #if int_conv1_weight_2[i][j] < 0:
        #    int_conv1_weight_2[i][j] += 256
        #if int_conv1_weight_3[i][j] < 0:
        #    int_conv1_weight_3[i][j] += 256

#for i in range(3):
 #   if int_conv1_bias[i] < 0:
  #          int_conv1_bias[i] += 256*256
#
#print ("Unsigned")
#print(int_conv1_weight_1)
#print(int_conv1_weight_2)
#print(int_conv1_weight_3)
#print(int_conv1_bias)
#np.savetxt('conv1_weight_1.mem', int_conv1_weight_1, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv1_weight_2.mem', int_conv1_weight_2, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv1_weight_3.mem', int_conv1_weight_3, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv1_bias.mem', int_conv1_bias, fmt='%1.2x',delimiter = " ")

tensor([[-0.0773,  0.2015,  0.1873, -0.0046,  0.1555],
        [-0.0696, -0.2170, -0.0775,  0.1051,  0.2148],
        [ 0.0901, -0.1994, -0.0626,  0.1994,  0.3410],
        [-0.0999, -0.0086, -0.0725,  0.1980,  0.2159],
        [ 0.0177, -0.1017, -0.1570, -0.0096,  0.2055]])
tensor([[-0.0953,  0.0423,  0.3730,  0.2413,  0.0263],
        [-0.1221,  0.1831,  0.2278,  0.3930,  0.2553],
        [ 0.1548,  0.3197,  0.5521,  0.8877,  0.4599],
        [ 0.0680,  0.5098,  0.8524,  0.5987,  0.1161],
        [ 0.1649,  0.1793,  0.4654,  0.3942,  0.2900]])
tensor([[ 0.2414,  0.0408, -0.2019,  0.0548, -0.1531],
        [ 0.1850,  0.3362,  0.5191,  0.5919,  0.4848],
        [-0.0308,  0.4638,  0.5315,  0.3755,  0.2551],
        [-0.1318, -0.3182, -0.1797, -0.0086,  0.0633],
        [-0.2279, -0.5504, -0.6544, -0.3949, -0.0072]])
weights
0xFF
0x03
0x02
0x00
0x02
0xFF
0xFD
0xFF
0x01
0x03
0x01
0xFD
0xFF
0x03
0x05
0xFF
0x00
0xFF
0x03
0x03
0x00
0xFF
0xFE
0x00
0x03
0xFF
0x00
0x05
0x03
0x00
0xFF
0x02
0x03

C:\Users\user\AppData\Local\Temp\ipykernel_33968\2863423040.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv1_weight_1 =  torch.tensor((model.conv1.weight.data[0][0]), dtype = torch.float32)
C:\Users\user\AppData\Local\Temp\ipykernel_33968\2863423040.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv1_weight_2 =  torch.tensor((model.conv1.weight.data[1][0]), dtype = torch.float32)
C:\Users\user\AppData\Local\Temp\ipykernel_33968\2863423040.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv1_weight_

In [7]:
############## Conv2 가중치 값 HEX 추출 ############
print(np.shape(model.conv2.weight))

# float => int

float_conv2_weight_11 =  torch.tensor((model.conv2.weight.data[0][0]), dtype = torch.float32)
float_conv2_weight_12 =  torch.tensor((model.conv2.weight.data[1][0]), dtype = torch.float32)
float_conv2_weight_13 =  torch.tensor((model.conv2.weight.data[2][0]), dtype = torch.float32)

float_conv2_weight_21 =  torch.tensor((model.conv2.weight.data[3][0]), dtype = torch.float32)
float_conv2_weight_22 =  torch.tensor((model.conv2.weight.data[4][0]), dtype = torch.float32)
float_conv2_weight_23 =  torch.tensor((model.conv2.weight.data[5][0]), dtype = torch.float32)

float_conv2_weight_31 =  torch.tensor((model.conv2.weight.data[6][0]), dtype = torch.float32)
float_conv2_weight_32 =  torch.tensor((model.conv2.weight.data[7][0]), dtype = torch.float32)
float_conv2_weight_33 =  torch.tensor((model.conv2.weight.data[8][0]), dtype = torch.float32)

float_conv2_weight_41 =  torch.tensor((model.conv2.weight.data[9][0]), dtype = torch.float32)
float_conv2_weight_42 =  torch.tensor((model.conv2.weight.data[10][0]), dtype = torch.float32)
float_conv2_weight_43 =  torch.tensor((model.conv2.weight.data[11][0]), dtype = torch.float32)

float_conv2_weight_51 =  torch.tensor((model.conv2.weight.data[12][0]), dtype = torch.float32)
float_conv2_weight_52 =  torch.tensor((model.conv2.weight.data[13][0]), dtype = torch.float32)
float_conv2_weight_53 =  torch.tensor((model.conv2.weight.data[14][0]), dtype = torch.float32)

float_conv2_weight_61 =  torch.tensor((model.conv2.weight.data[15][0]), dtype = torch.float32)
float_conv2_weight_62 =  torch.tensor((model.conv2.weight.data[16][0]), dtype = torch.float32)
float_conv2_weight_63 =  torch.tensor((model.conv2.weight.data[17][0]), dtype = torch.float32)


#print ("Signed")
print(float_conv2_weight_11)
print(float_conv2_weight_12)
print(float_conv2_weight_13)
print(float_conv2_weight_21)
print(float_conv2_weight_22)
print(float_conv2_weight_23)
print(float_conv2_weight_31)
print(float_conv2_weight_32)
print(float_conv2_weight_33)
print(float_conv2_weight_41)
print(float_conv2_weight_42)
print(float_conv2_weight_43)
print(float_conv2_weight_51)
print(float_conv2_weight_52)
print(float_conv2_weight_53)
print(float_conv2_weight_61)
print(float_conv2_weight_62)
print(float_conv2_weight_63)

float_conv2_weight = [[ 0.1184,  0.1777, -0.0042, -0.1301, -0.1472],
        [-0.1608, -0.1476,  0.1198,  0.0973,  0.0907],
        [ 0.1599, -0.0432,  0.1043, -0.1840,  0.0241],
        [-0.1670, -0.0661, -0.0222, -0.1046, -0.0475],
        [ 0.1665,  0.0124, -0.1940,  0.0043, -0.1785],
[ 0.1317, -0.0243,  0.1257,  0.2123, -0.0665],
        [-0.1901,  0.0699,  0.2054,  0.0943,  0.1152],
        [-0.0420,  0.0330,  0.2671, -0.0268,  0.1289],
        [ 0.1097,  0.1125,  0.0085,  0.1843, -0.1247],
        [ 0.1298,  0.1431,  0.0524,  0.1489,  0.1840],
[-0.2198,  0.0707,  0.2381,  0.0219, -0.1041],
        [-0.1509,  0.1441,  0.1081,  0.1325, -0.1573],
        [-0.0587, -0.0417,  0.2373,  0.0116, -0.0945],
        [-0.0157, -0.0278, -0.0492, -0.0701,  0.1609],
        [-0.0615, -0.0781, -0.0163,  0.2394,  0.1353],
[-0.0962,  0.0199,  0.2266, -0.1618, -0.0050],
        [-0.0727,  0.0810,  0.1946, -0.1478, -0.0307],
        [ 0.0315,  0.1458, -0.0004, -0.0993, -0.1629],
        [ 0.1195, -0.1430,  0.1795, -0.1107, -0.0704],
        [-0.0971, -0.1640,  0.0645,  0.1642,  0.0202],
[ 0.0505,  0.0796,  0.0385,  0.0586, -0.1782],
        [-0.0611, -0.0952, -0.2513, -0.0485,  0.2246],
        [-0.0773, -0.1652, -0.0665,  0.0073,  0.2135],
        [-0.0557, -0.0851, -0.2560, -0.2050, -0.0694],
        [ 0.1011, -0.1036,  0.1115,  0.0099, -0.1640],
[ 0.1290, -0.1563,  0.1341,  0.1788, -0.0821],
        [-0.1473, -0.1850,  0.1907,  0.1456,  0.1235],
        [ 0.0419,  0.0074,  0.0123,  0.0296,  0.0577],
        [-0.1207,  0.0259, -0.1164, -0.0834, -0.0293],
        [ 0.0425, -0.1879, -0.1615, -0.0523,  0.1819],
[-0.2999, -0.2512, -0.3695, -0.3287, -0.1825],
        [ 0.0784,  0.1102,  0.0060, -0.1921, -0.0706],
        [-0.0723,  0.3526,  0.2258,  0.2869,  0.2250],
        [-0.0418,  0.0080, -0.1967, -0.2840,  0.1730],
        [-0.0536, -0.0408,  0.0718, -0.1427,  0.0544],
[-0.1292, -0.1552,  0.1568, -0.0277, -0.1359],
        [-0.0848,  0.2838,  0.1847, -0.3486, -0.1957],
        [ 0.1465,  0.1529, -0.2023, -0.2469,  0.2335],
        [-0.1802,  0.1978, -0.0155, -0.1185,  0.0976],
        [-0.0020, -0.0766,  0.2636, -0.0435,  0.3286],
[ 0.1341, -0.2531, -0.3342, -0.2920, -0.1205],
        [-0.2182, -0.0063, -0.0346, -0.2450, -0.1592],
        [ 0.1975,  0.3989,  0.0252, -0.2345,  0.0569],
        [ 0.1997, -0.1044, -0.0140, -0.1917,  0.0346],
        [-0.0273, -0.1904, -0.1436,  0.1506,  0.0546],
[ 0.1614, -0.0780,  0.1382,  0.1170, -0.0392],
        [ 0.0955,  0.1066,  0.0379, -0.2111, -0.1429],
        [ 0.1930,  0.0045, -0.3711, -0.2764, -0.2159],
        [ 0.2218, -0.0204, -0.2184,  0.0313, -0.0597],
        [ 0.1819,  0.0733,  0.1990, -0.1016,  0.2146],
[ 0.0509,  0.2499,  0.2172,  0.0388,  0.1408],
        [ 0.0590,  0.0518,  0.0766,  0.3883,  0.1145],
        [-0.3542, -0.4204,  0.0768,  0.2111, -0.0452],
        [-0.1748, -0.5332, -0.2188,  0.3843, -0.0563],
        [-0.0901, -0.0735, -0.0298,  0.2214, -0.1340],
[ 0.2469,  0.2628,  0.1577,  0.2869,  0.0428],
        [-0.0219,  0.1019,  0.1583, -0.1550, -0.0962],
        [-0.2320, -0.4619, -0.6237, -0.5795, -0.1699],
        [-0.3245, -0.2049, -0.0837,  0.1662,  0.0400],
        [-0.0076,  0.2074,  0.2341,  0.3290, -0.0857],
[-0.0383,  0.2112,  0.3010, -0.0745,  0.0316],
        [ 0.2390,  0.2874, -0.0581,  0.0664,  0.1628],
        [-0.1148,  0.0006, -0.0919,  0.2606, -0.0914],
        [-0.0016,  0.2269,  0.2143, -0.0500, -0.1143],
        [ 0.1794,  0.3036,  0.1181, -0.1664,  0.1184],
[ 0.1319,  0.0015,  0.1172,  0.0875,  0.0030],
        [-0.0456,  0.0598, -0.1365, -0.0250, -0.1217],
        [-0.1414, -0.1543,  0.0386, -0.0426, -0.2418],
        [ 0.1072, -0.2245,  0.1625,  0.0802,  0.1022],
        [-0.1416, -0.1283, -0.1646, -0.0346,  0.0324],
[-0.0181,  0.1153, -0.0137,  0.1152, -0.0995],
        [-0.1205,  0.2284,  0.3136,  0.4492,  0.2332],
        [-0.1927,  0.2196, -0.0120, -0.0824,  0.0420],
        [-0.1457,  0.0015, -0.1079, -0.1756, -0.1657],
        [ 0.0151,  0.1027,  0.2652,  0.0101, -0.1275],
[-0.2177, -0.2263, -0.1631, -0.2365, -0.1516],
        [-0.1785, -0.2252, -0.5882, -0.6372, -0.1528],
        [ 0.1287, -0.2361, -0.2780, -0.1335, -0.1741],
        [ 0.1401,  0.0814,  0.2366, -0.1604,  0.1257],
        [ 0.2126,  0.0126,  0.0569,  0.0815,  0.2374],
[-0.0626, -0.1978, -0.3155,  0.0076, -0.2358],
        [-0.1150, -0.1860, -0.1308, -0.2000, -0.2655],
        [-0.1086,  0.0929,  0.0455, -0.0445, -0.0717],
        [-0.1809,  0.0684,  0.0342,  0.2559,  0.2426],
        [-0.2472, -0.3982, -0.1720,  0.1056,  0.1337],
[-0.2339, -0.1244,  0.0774,  0.2545, -0.1562],
        [-0.0712, -0.2176, -0.1240,  0.1255, -0.1296],
        [ 0.2199, -0.0355,  0.2808, -0.0167,  0.1841],
        [-0.0695,  0.1678,  0.1806, -0.0723, -0.0707],
        [ 0.0850,  0.2393, -0.0052,  0.1778,  0.1152]]




# signed int => unsigned int
from fxpmath import Fxp
for i in range(90):
    for j in range(5):
        
        x = Fxp(float_conv2_weight[i][j], signed=True, n_word=8, n_frac=4)
        #print(x)         # float value
        #print(x.bin())   # binary representation
        print(x.hex())
        #print(x.val)








torch.Size([18, 1, 5, 5])
tensor([[ 0.1184,  0.1777, -0.0042, -0.1301, -0.1472],
        [-0.1608, -0.1476,  0.1198,  0.0973,  0.0907],
        [ 0.1599, -0.0432,  0.1043, -0.1840,  0.0241],
        [-0.1670, -0.0661, -0.0222, -0.1046, -0.0475],
        [ 0.1665,  0.0124, -0.1940,  0.0043, -0.1785]])
tensor([[ 0.1317, -0.0243,  0.1257,  0.2123, -0.0665],
        [-0.1901,  0.0699,  0.2054,  0.0943,  0.1152],
        [-0.0420,  0.0330,  0.2671, -0.0268,  0.1289],
        [ 0.1097,  0.1125,  0.0085,  0.1843, -0.1247],
        [ 0.1298,  0.1431,  0.0524,  0.1489,  0.1840]])
tensor([[-0.2198,  0.0707,  0.2381,  0.0219, -0.1041],
        [-0.1509,  0.1441,  0.1081,  0.1325, -0.1573],
        [-0.0587, -0.0417,  0.2373,  0.0116, -0.0945],
        [-0.0157, -0.0278, -0.0492, -0.0701,  0.1609],
        [-0.0615, -0.0781, -0.0163,  0.2394,  0.1353]])
tensor([[-0.0962,  0.0199,  0.2266, -0.1618, -0.0050],
        [-0.0727,  0.0810,  0.1946, -0.1478, -0.0307],
        [ 0.0315,  0.1458, -0.0004, 

C:\Users\user\AppData\Local\Temp\ipykernel_33968\3506092626.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv2_weight_11 =  torch.tensor((model.conv2.weight.data[0][0]), dtype = torch.float32)
C:\Users\user\AppData\Local\Temp\ipykernel_33968\3506092626.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv2_weight_12 =  torch.tensor((model.conv2.weight.data[1][0]), dtype = torch.float32)
C:\Users\user\AppData\Local\Temp\ipykernel_33968\3506092626.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv2_weight_1

In [10]:
############## Conv3 가중치 값 HEX 추출 ############

print(np.shape(model.conv3.weight))
# float => int
#float_conv3_bias = torch.tensor((model.conv3.bias.data), dtype = torch.float32)

float_conv3_weight_11 =  torch.tensor((model.conv3.weight.data[0][0]), dtype = torch.float32)
float_conv3_weight_12 =  torch.tensor((model.conv3.weight.data[0][1]), dtype = torch.float32)
float_conv3_weight_13 =  torch.tensor((model.conv3.weight.data[0][2]), dtype = torch.float32)

float_conv3_weight_21 =  torch.tensor((model.conv3.weight.data[1][0]), dtype = torch.float32)
float_conv3_weight_22 =  torch.tensor((model.conv3.weight.data[1][1]), dtype = torch.float32)
float_conv3_weight_23 =  torch.tensor((model.conv3.weight.data[1][2]), dtype = torch.float32)

float_conv3_weight_31 =  torch.tensor((model.conv3.weight.data[2][0]), dtype = torch.float32)
float_conv3_weight_32 =  torch.tensor((model.conv3.weight.data[2][1]), dtype = torch.float32)
float_conv3_weight_33 =  torch.tensor((model.conv3.weight.data[2][2]), dtype = torch.float32)

float_conv3_weight_41 =  torch.tensor((model.conv3.weight.data[3][0]), dtype = torch.float32)
float_conv3_weight_42 =  torch.tensor((model.conv3.weight.data[3][1]), dtype = torch.float32)
float_conv3_weight_43 =  torch.tensor((model.conv3.weight.data[3][2]), dtype = torch.float32)

float_conv3_weight_51 =  torch.tensor((model.conv3.weight.data[4][0]), dtype = torch.float32)
float_conv3_weight_52 =  torch.tensor((model.conv3.weight.data[4][1]), dtype = torch.float32)
float_conv3_weight_53 =  torch.tensor((model.conv3.weight.data[4][2]), dtype = torch.float32)

float_conv3_weight_61 =  torch.tensor((model.conv3.weight.data[5][0]), dtype = torch.float32)
float_conv3_weight_62 =  torch.tensor((model.conv3.weight.data[5][1]), dtype = torch.float32)
float_conv3_weight_63 =  torch.tensor((model.conv3.weight.data[5][2]), dtype = torch.float32)


#print ("Signed")
#print(float_conv3_bias)
print(float_conv3_weight_11)
print(float_conv3_weight_12)
print(float_conv3_weight_13)
print(float_conv3_weight_21)
print(float_conv3_weight_22)
print(float_conv3_weight_23)
print(float_conv3_weight_31)
print(float_conv3_weight_32)
print(float_conv3_weight_33)
print(float_conv3_weight_41)
print(float_conv3_weight_42)
print(float_conv3_weight_43)
print(float_conv3_weight_51)
print(float_conv3_weight_52)
print(float_conv3_weight_53)
print(float_conv3_weight_61)
print(float_conv3_weight_62)
print(float_conv3_weight_63)

#float_conv3_bias = [[ 0.1263, -0.4903,  0.1191]]
float_conv3_weight3 = [[0.1435],
[0.0478],
[0.2155],
[-0.0104],
[-0.0491],
[-0.0628],
[0.0053],
[0.3131],
[0.3038],
[-0.0861],
[0.0614],
[0.1344],
[-0.0841],
[0.0324],
[-0.1783],
[0.1439],
[0.1799],
[-0.1860]]
from fxpmath import Fxp

#for m in range(1):
#    for n in range(3):
#        b = Fxp(float_conv3_bias[m][n], signed=True, n_word=16, n_frac=8
   
        #print(b)         # float value
        #print(b.bin())   # binary representation
#        print(b.hex())
        #print(b.val)
        


for i in range(18):
    for j in range(1):
        
        x = Fxp(float_conv3_weight3[i][j], signed=True, n_word=8, n_frac=4)
        #print(x)         # float value
        #print(x.bin())   # binary representation
        print(x.hex())
        #print(x.val)
# signed int => unsigned int


#for i in range(3):
#    if int_conv3_bias[i] < 0:
#            int_conv3_bias[i] += 256*256



#np.savetxt('conv3_weight_11.txt', float_conv3_weight_11, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv3_weight_12.txt', float_conv3_weight_12, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv3_weight_13.txt', float_conv3_weight_13, fmt='%1.2x',delimiter = " ")

#np.savetxt('conv3_weight_21.txt', float_conv3_weight_21, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv3_weight_22.txt', float_conv3_weight_22, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv3_weight_23.txt', float_conv3_weight_23, fmt='%1.2x',delimiter = " ")

#np.savetxt('conv3_weight_31.txt', float_conv3_weight_31, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv3_weight_32.txt', float_conv3_weight_32, fmt='%1.2x',delimiter = " ")
#np.savetxt('conv3_weight_33.txt', float_conv3_weight_33, fmt='%1.2x',delimiter = " ")

#np.savetxt('conv3_bias.txt', int_conv3_bias, fmt='%1.2x',delimiter = " ")

torch.Size([6, 18, 1, 1])
tensor([[0.1435]])
tensor([[0.0478]])
tensor([[0.2155]])
tensor([[-0.0104]])
tensor([[-0.0491]])
tensor([[-0.0628]])
tensor([[0.0053]])
tensor([[0.3131]])
tensor([[0.3038]])
tensor([[-0.0861]])
tensor([[0.0614]])
tensor([[0.1344]])
tensor([[-0.0841]])
tensor([[0.0324]])
tensor([[-0.1783]])
tensor([[0.1439]])
tensor([[0.1799]])
tensor([[-0.1860]])
0x02
0x00
0x03
0x00
0x00
0xFF
0x00
0x05
0x04
0xFF
0x00
0x02
0xFF
0x00
0xFE
0x02
0x02
0xFE


C:\Users\user\AppData\Local\Temp\ipykernel_33968\833655838.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv3_weight_11 =  torch.tensor((model.conv3.weight.data[0][0]), dtype = torch.float32)
C:\Users\user\AppData\Local\Temp\ipykernel_33968\833655838.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv3_weight_12 =  torch.tensor((model.conv3.weight.data[0][1]), dtype = torch.float32)
C:\Users\user\AppData\Local\Temp\ipykernel_33968\833655838.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  float_conv3_weight_13 =

In [11]:
############## FC Layer가중치 값 HEX 추출 ############

##(np.shape(model.fc_1.weight))
#print((model.fc_1.weight * 128).int())

#print(np.shape(model.fc_1.bias))


float_fc_weight = (model.fc_1.weight).float()
float_fc_bias = (model.fc_1.bias).float()
print(float_fc_bias)
print(float_fc_weight)
fc_weight = [[-6.6225e-02,  3.2556e-02,  5.5242e-02,  6.8833e-02,  8.6716e-02,
          1.4086e-01,  8.5182e-02,  1.1414e-01, -3.7614e-02, -2.3148e-02,
         -9.5934e-02,  7.5057e-02, -4.4981e-02,  1.3153e-01, -1.3041e-01,
         -1.1367e-01, -9.4842e-03, -5.3155e-02,  8.4044e-02, -2.3950e-02,
         -1.0267e-01, -6.4267e-02,  1.5496e-01,  6.7653e-02, -3.3393e-01,
         -2.6816e-01, -1.0517e-01, -7.7368e-02, -6.3888e-02, -3.5167e-02,
         -3.7750e-02,  1.2781e-01,  3.3492e-02, -3.4174e-02,  5.9913e-02,
          1.1882e-01,  1.5604e-02,  9.1628e-02, -1.2629e-02,  2.2870e-01,
          9.1511e-02, -1.4586e-01,  8.3657e-02,  2.3657e-01,  2.4047e-01,
         -1.1409e-01, -1.7323e-01,  4.7743e-03, -2.0424e-02,  4.9587e-02,
          4.1078e-02,  2.3212e-01,  1.3312e-01,  1.3742e-01, -4.9071e-02,
          6.2087e-02, -5.6849e-02, -1.0033e-01, -2.2385e-01, -3.1123e-01,
         -1.0135e-01,  1.0084e-01,  2.0027e-01, -3.5494e-02,  1.0980e-02,
          1.3104e-01,  6.6422e-02, -7.0186e-02,  7.2191e-02,  3.9484e-02,
          8.4425e-02, -8.9432e-02, -8.4961e-02, -8.4606e-02,  5.5238e-02,
          4.5593e-02, -1.6321e-01,  8.2383e-02, -1.0513e-02,  8.7935e-02,
         -5.5364e-02,  1.0673e-01, -7.1496e-02, -7.9954e-02,  4.6827e-02,
         -7.9740e-02, -2.3537e-01, -1.1592e-01,  8.0778e-02, -2.6015e-01,
          5.3951e-02,  2.2513e-01,  8.5862e-02, -3.2028e-02,  1.8300e-01,
          5.5398e-02],
        [-8.4095e-02,  6.3263e-02,  1.6329e-01,  2.3265e-02, -5.9549e-03,
         -6.2045e-02,  1.7890e-02,  7.8671e-02,  1.1074e-01,  2.8243e-03,
         -3.2255e-02, -2.5943e-02, -4.5984e-02,  1.0443e-01,  2.6870e-02,
         -8.8618e-03, -2.6972e-02, -1.6184e-01, -2.5233e-01, -2.8135e-01,
         -6.0017e-02,  1.0854e-01, -2.7072e-01, -1.6214e-01, -1.5560e-01,
         -5.0585e-02, -6.2610e-02, -4.6266e-03, -2.3314e-01,  2.1398e-02,
          4.2059e-02,  5.1598e-02, -2.6564e-02,  1.6243e-01,  2.5057e-01,
          5.6381e-02, -1.0354e-01,  8.7524e-02,  1.4860e-01, -1.2677e-01,
         -9.9082e-02,  2.5347e-01,  3.8864e-02,  5.7119e-02, -1.0164e-01,
          1.1912e-01,  1.0576e-01,  5.5529e-02, -1.0382e-02, -1.5697e-01,
          9.9296e-02,  1.7580e-02,  1.1084e-01,  5.7855e-02,  2.1289e-01,
         -9.0302e-02, -4.7572e-02,  7.4237e-02,  2.4430e-01, -5.2952e-02,
          1.6876e-01,  8.0620e-02, -7.3173e-02,  2.7215e-01,  9.4167e-02,
         -1.6603e-03, -1.4635e-01,  1.2108e-02, -1.2075e-01,  1.3089e-01,
          1.5438e-01, -1.7972e-02,  9.9362e-02, -1.2370e-01, -3.5695e-02,
         -1.7482e-03,  1.4912e-01,  8.5569e-02,  1.9617e-01,  1.7401e-01,
          2.8788e-02, -4.5917e-03,  1.1330e-01,  8.4029e-02, -8.7452e-02,
          2.5682e-01, -2.5958e-01, -3.0242e-01,  5.5701e-02, -2.7349e-02,
         -1.2907e-01, -2.9114e-01,  7.3984e-02,  1.5232e-01, -4.9229e-01,
          1.2103e-01],
        [ 3.7664e-02, -1.2190e-02,  9.6625e-02,  6.2210e-02, -1.1727e-01,
         -1.1562e-01,  2.0475e-01,  4.3782e-02, -1.2909e-01, -2.8440e-02,
         -2.9444e-02, -1.9192e-01,  1.9533e-02, -6.8360e-03, -3.4114e-02,
          2.5344e-02,  9.4265e-02,  1.0698e-01,  1.0703e-01,  6.3482e-03,
          4.3164e-01,  1.0142e-01,  4.1497e-02,  2.1196e-01, -5.6719e-02,
         -9.3606e-02, -8.5046e-02, -1.7235e-01, -2.9160e-01,  4.7828e-02,
         -2.5945e-02,  2.4157e-01, -1.2213e-01, -2.0752e-02,  9.7038e-02,
          3.4821e-02, -1.8796e-01, -2.0718e-01,  9.7396e-02,  1.1684e-01,
         -9.9320e-02, -4.9574e-02,  1.5355e-01, -3.1482e-01,  1.9748e-01,
         -8.9494e-02, -1.3103e-01,  2.0197e-03, -1.8131e-01, -3.5504e-03,
          1.3379e-01,  5.5091e-02,  1.3466e-01,  1.2647e-01,  7.1849e-02,
         -2.4761e-02,  3.7153e-01,  3.8314e-02, -6.3391e-02, -1.3208e-01,
         -1.1597e-01, -8.6217e-02,  1.3040e-02,  3.4986e-01,  2.1997e-02,
         -6.1772e-02,  3.9376e-02, -3.2735e-01,  2.1495e-01, -3.5137e-02,
         -1.6507e-01, -1.1525e-02,  4.4876e-02,  1.1829e-02, -8.2609e-03,
          2.7066e-01,  5.8643e-02, -4.9348e-02, -2.5994e-02,  3.0485e-01,
          8.7909e-03, -4.9846e-02, -1.5638e-01, -2.8386e-01, -2.4185e-02,
         -1.6744e-02, -1.3982e-03, -2.8604e-02,  2.1444e-01,  1.0324e-01,
          5.0863e-02, -7.6575e-02,  2.4376e-01,  8.4093e-02, -1.6324e-01,
          2.2216e-01],
        [-4.0470e-02,  3.2170e-02,  7.3006e-02,  8.6206e-02,  4.1420e-02,
         -3.8547e-02,  2.4198e-02, -8.4722e-02,  1.0197e-02,  8.7263e-02,
         -1.2745e-01, -1.9789e-04,  2.6842e-02,  1.3672e-02,  8.5757e-02,
         -6.1096e-02,  2.2066e-01,  2.5827e-01, -6.0008e-03, -1.2356e-01,
          3.6031e-01,  3.2153e-02,  3.0802e-02, -1.8758e-01, -3.6237e-02,
          3.1618e-02,  8.6423e-02,  2.3901e-02,  2.2253e-01,  1.2660e-01,
         -2.3803e-02, -1.3003e-01, -3.4190e-01, -2.2100e-01,  4.4635e-02,
          1.7485e-01, -5.4655e-02,  2.0100e-01, -1.2380e-01, -2.6847e-01,
         -1.0482e-02,  1.7076e-01, -2.6683e-01, -5.2096e-02, -2.9916e-01,
         -2.7215e-01, -1.9875e-02, -2.2110e-02, -9.6906e-02, -5.0813e-02,
         -3.0514e-03, -1.4289e-01, -1.2275e-01, -2.1364e-01, -1.5910e-01,
         -1.8805e-01, -9.3259e-03, -1.4689e-01, -1.8086e-01,  1.6982e-01,
          1.9228e-01,  1.1483e-01, -1.0245e-01, -5.3320e-02,  3.1561e-02,
          3.0372e-02,  1.0566e-01, -9.4280e-02,  4.8850e-02, -1.8293e-02,
          7.4666e-02, -1.5537e-01,  1.1456e-01,  1.4605e-01,  5.2352e-03,
         -9.2179e-02,  1.3659e-01, -2.5743e-02, -1.1507e-01, -2.2098e-01,
         -1.0834e-01,  1.0111e-01,  7.5301e-02, -3.6238e-02,  4.9994e-02,
          2.3608e-01,  3.3682e-01,  2.2014e-01, -1.3645e-01, -1.1892e-01,
          5.4488e-02, -3.4124e-02, -1.6638e-01,  6.7973e-02,  3.8814e-01,
          3.3219e-02],
        [ 1.1308e-01,  8.2292e-02,  1.9947e-02,  7.4375e-02, -2.9953e-02,
         -1.1148e-01, -1.7058e-01, -1.5541e-01, -3.2702e-02, -5.6337e-02,
         -7.1867e-02, -3.5793e-02, -1.3938e-02, -2.5947e-02, -3.7676e-02,
          1.5220e-02,  2.7737e-02, -3.2479e-01, -3.6800e-01, -1.6443e-01,
         -1.9411e-01, -2.9360e-01, -2.3616e-01, -1.7911e-01,  9.9358e-02,
          7.1145e-02,  4.3900e-02,  5.9845e-02,  2.1982e-01,  1.0244e-01,
          1.1474e-01,  9.6551e-02,  2.4892e-01,  1.5009e-01,  3.1459e-01,
          2.7056e-01,  1.2293e-01,  5.0492e-02,  2.0291e-01,  1.6220e-01,
         -1.1057e-01, -3.9266e-02,  4.2088e-02, -1.4810e-01, -2.9828e-02,
          2.0666e-01,  8.4155e-02,  2.3866e-02,  1.1577e-02,  1.3587e-01,
         -1.5831e-02, -7.2568e-03,  8.8296e-03,  1.1342e-01, -1.7735e-01,
         -2.4566e-03, -2.8070e-02,  8.8844e-02,  2.1838e-01,  2.2268e-01,
          5.8904e-03,  8.9845e-02, -8.0377e-02, -2.2781e-01, -1.3939e-01,
         -2.7587e-01, -2.7923e-01, -1.6532e-01, -8.0520e-02, -3.0224e-02,
          3.9469e-03, -2.8205e-02,  1.9692e-02,  6.3267e-02,  8.2390e-02,
         -4.6643e-02,  2.7274e-02, -1.2639e-01,  7.1896e-02,  4.0444e-02,
         -5.0740e-02, -7.1817e-02,  2.0229e-01,  1.3291e-01,  1.9402e-01,
         -3.9710e-02,  6.7428e-02, -3.2232e-02, -1.3801e-01, -1.6778e-01,
          1.5196e-01, -5.1015e-02, -1.1681e-01, -9.7400e-02, -3.2690e-01,
         -1.8869e-02],
        [ 7.0306e-02,  6.1576e-02, -1.8232e-01, -1.7878e-01,  1.3253e-02,
          2.3962e-02, -1.3051e-01, -4.0697e-02, -6.2435e-02, -7.1635e-03,
          2.7667e-02,  1.9461e-01,  2.0451e-02,  8.9417e-02,  1.1847e-01,
         -5.3932e-02, -1.2022e-01,  1.1547e-02,  1.9381e-01,  1.5139e-01,
         -5.4729e-02,  7.7917e-02, -7.4178e-02,  2.0149e-01,  1.4637e-01,
          8.5925e-02,  1.2594e-01,  6.4392e-02,  3.2194e-01, -1.5965e-02,
          8.4646e-02, -1.0490e-01,  7.9911e-02,  6.7037e-03, -2.9646e-01,
         -4.2846e-01,  2.2150e-01,  1.5781e-02, -3.7039e-01, -4.2994e-01,
         -3.0347e-02, -1.1471e-01,  3.8932e-02,  8.9337e-02, -9.4180e-02,
         -1.2295e-01, -6.8640e-02,  1.6956e-01, -1.3404e-02,  2.3860e-02,
         -2.0042e-02, -1.9561e-01, -9.8455e-02, -2.1969e-02,  1.0084e-01,
          9.7036e-02,  1.4615e-01,  4.7965e-02,  6.6582e-02,  8.1162e-02,
          1.6618e-01, -8.3714e-02, -4.5869e-02,  6.1326e-02, -9.2739e-02,
          8.3452e-02,  2.3759e-01,  4.0640e-01, -3.6018e-02, -6.3108e-02,
          1.3316e-02,  1.6075e-01,  9.0032e-02,  1.1810e-03, -6.5371e-02,
         -1.0350e-01,  3.0301e-01,  5.2208e-02, -1.5181e-01, -1.1825e-01,
         -1.2844e-01, -9.7420e-02, -2.0221e-02,  1.9430e-01, -1.3931e-01,
         -1.4453e-02, -2.1196e-01, -2.1577e-01, -2.5744e-02, -1.5737e-01,
         -9.2890e-02, -2.2314e-01, -4.0968e-02,  1.8771e-01,  2.2581e-01,
          5.7778e-02],
        [-4.7018e-02,  1.4302e-01, -4.4327e-02, -1.3050e-01, -6.3246e-02,
         -3.3748e-02, -2.9534e-02, -2.7970e-02,  1.0837e-01,  1.5011e-02,
         -4.6480e-02,  4.0015e-02, -5.1142e-02,  6.3716e-03,  8.8488e-03,
         -1.2526e-02, -2.4487e-01, -3.3652e-01, -2.1464e-01,  3.4777e-01,
         -2.4318e-01, -2.8408e-01,  7.7136e-03,  2.7977e-01, -3.0212e-01,
         -1.0295e-01, -4.7130e-02, -1.9075e-01,  4.6135e-02, -1.4775e-01,
          9.5834e-02, -1.9962e-03,  1.2141e-01,  3.9361e-02,  7.2808e-02,
         -2.5940e-01,  9.2774e-03,  1.8890e-02, -2.4703e-01, -1.3669e-01,
          1.0613e-01,  6.1018e-02, -1.3803e-01,  8.9025e-02,  3.1226e-01,
          1.0175e-01, -9.8680e-02, -6.1244e-02, -5.2437e-02, -1.4456e-01,
         -1.3720e-01, -1.6016e-01, -4.5704e-02,  4.8326e-02,  1.4195e-01,
          2.6608e-01, -9.2636e-02,  1.4359e-01,  2.8097e-02, -4.2358e-02,
         -2.0102e-01, -1.3536e-01,  1.0053e-01,  1.7321e-02, -6.5864e-02,
         -1.5057e-01,  7.4170e-02,  6.1726e-02, -5.4822e-02, -4.1805e-02,
          2.0091e-01,  3.2085e-01, -1.5857e-01, -3.0637e-02,  2.7448e-02,
         -3.8163e-02, -1.8654e-01,  7.7815e-02,  1.9332e-02, -4.6774e-02,
          5.4963e-03, -7.2700e-03, -4.6082e-02, -1.3000e-02,  1.7182e-01,
         -2.3271e-01, -2.5716e-01, -1.0892e-01,  6.2360e-02,  2.8512e-01,
         -1.5542e-01,  3.2691e-01,  3.1535e-02,  3.9158e-02,  3.5225e-01,
          3.0335e-01],
        [ 2.0869e-02,  1.3971e-01,  1.2722e-01,  5.9499e-02,  7.9136e-02,
          3.2876e-02,  6.3345e-02, -1.0381e-01,  7.0327e-02, -5.6021e-02,
          4.7694e-02, -1.6968e-01,  3.7234e-02,  5.0338e-02, -1.8386e-02,
         -1.2451e-01,  2.1683e-01,  1.2961e-01,  5.9455e-02, -5.5498e-02,
          2.8876e-01,  1.4261e-01,  1.8487e-01, -9.5894e-02,  2.4031e-01,
         -8.3247e-02, -1.6296e-02,  5.4169e-02, -2.9342e-02, -2.6003e-01,
          2.8941e-02,  1.6257e-01, -5.3330e-02, -4.9601e-02, -1.4858e-03,
         -1.3697e-01, -1.5934e-01, -7.5391e-02,  3.2852e-01,  1.2177e-01,
          8.7232e-02,  9.4942e-02,  2.7874e-01,  1.3071e-01, -1.8636e-01,
          2.8442e-01,  1.0103e-01, -1.9614e-01,  7.8748e-02,  1.8730e-01,
          1.2427e-01,  1.6539e-01, -1.5243e-02,  2.1468e-01,  2.8279e-02,
         -6.2101e-02, -1.3047e-01, -1.0014e-01,  1.8369e-01,  1.3831e-01,
         -5.8661e-02,  1.1693e-01, -9.1601e-02, -3.0088e-01, -9.1717e-03,
          1.1009e-01,  7.9640e-02,  1.3871e-01,  7.3086e-04, -1.0700e-01,
          1.7048e-01,  5.9855e-02,  2.8726e-01,  1.2809e-01, -1.5109e-01,
          7.2226e-02, -8.8942e-02,  1.4850e-02,  1.6172e-02, -7.1029e-03,
         -2.4776e-01, -2.4779e-01, -1.2215e-01, -4.2312e-02, -1.3449e-01,
          6.6630e-02, -1.1936e-01,  3.1432e-02,  1.8863e-01, -1.8296e-01,
         -1.1521e-01, -2.9818e-03, -8.0898e-02, -9.3964e-02, -4.4481e-01,
         -3.7265e-01],
        [-1.0915e-01,  6.0487e-03, -2.6825e-01,  6.0598e-02, -2.4549e-02,
          8.8001e-02, -2.4926e-02, -1.2354e-01,  3.0695e-02,  7.9460e-02,
         -9.7738e-02, -3.3442e-02, -5.9938e-02,  7.3952e-02, -2.8283e-02,
          2.1566e-01, -8.4645e-02,  1.7345e-01,  2.8290e-02,  7.3085e-02,
         -7.0060e-02, -5.4586e-02, -7.1739e-02, -9.9966e-02,  1.2217e-01,
          2.4147e-01, -8.9706e-02,  2.0175e-01, -3.4084e-01,  1.6492e-01,
         -2.5207e-02, -1.4302e-01,  1.4863e-01,  1.0918e-01, -1.1954e-01,
         -2.8527e-02, -5.9190e-03, -1.1985e-01, -2.6616e-01,  3.9518e-02,
         -4.7514e-02,  3.1681e-03, -1.4639e-01, -1.1669e-01,  2.8215e-01,
         -8.4671e-02, -1.5599e-01, -1.0251e-02,  4.0808e-02, -1.6813e-01,
         -1.3070e-01,  1.0655e-02,  5.2427e-02, -2.2039e-01,  8.3956e-02,
         -1.7240e-01,  9.6387e-02, -3.7873e-02,  5.4630e-02, -2.1626e-02,
         -1.2024e-01, -3.7603e-01, -6.9198e-02,  1.9696e-02,  7.6373e-02,
          8.2508e-02,  1.3725e-01, -2.6848e-02, -2.6020e-01, -6.3209e-02,
         -5.6235e-02, -1.9267e-02, -1.3467e-01, -7.5441e-02,  2.9640e-02,
         -1.5758e-02, -3.4012e-02, -2.0289e-01, -1.5440e-01, -1.5100e-01,
          2.7721e-01,  5.4123e-03,  1.4862e-01,  1.6594e-01, -4.2599e-02,
         -6.9132e-02,  2.1144e-01,  3.0043e-01,  1.2972e-01,  3.3903e-01,
         -1.0348e-01,  6.1745e-03,  6.2743e-02, -1.0543e-01,  2.4746e-01,
          5.3212e-02],
        [-1.5756e-01, -1.5974e-01, -1.0484e-01,  1.4071e-01, -1.0550e-01,
         -3.1597e-02,  8.8742e-02,  1.6610e-01, -4.4592e-02,  2.2722e-02,
         -5.5368e-02,  8.3352e-02,  6.4819e-02, -3.4194e-02, -1.5154e-02,
         -6.0946e-02, -5.5572e-02,  2.5791e-02,  2.1878e-01,  1.1456e-02,
         -3.1748e-01, -1.5705e-01,  8.7747e-04,  9.1801e-02,  8.3374e-02,
          1.9374e-01,  1.0021e-01, -2.8527e-02,  3.2867e-01,  1.5340e-01,
         -1.1741e-01, -2.0426e-01, -8.9856e-04, -1.6089e-01, -2.9962e-01,
         -1.1050e-01,  6.5009e-02, -1.3030e-02, -4.9626e-02,  1.8599e-01,
         -3.2962e-03, -6.2939e-02,  1.3340e-01,  1.0585e-01, -1.4301e-01,
          5.3546e-02,  1.1987e-01,  2.4238e-01,  3.1716e-02, -1.2747e-01,
          1.5714e-02,  1.1639e-01, -1.3231e-01, -3.2741e-01, -4.6124e-02,
          1.1899e-01, -2.0949e-01,  1.3598e-01, -2.0763e-01, -7.2700e-02,
         -3.2497e-03,  1.6921e-01, -1.2114e-01, -3.4188e-01,  1.2861e-01,
          1.5374e-01,  3.4947e-02,  6.9252e-03,  4.5859e-02,  1.0792e-03,
         -2.4421e-01, -3.7357e-01, -2.2682e-02,  9.4811e-03,  6.7646e-02,
         -1.7351e-01, -7.8633e-02,  9.0672e-02,  1.6199e-01,  3.8564e-03,
          8.4979e-02,  2.3677e-01,  5.5735e-02, -8.2782e-02, -1.2861e-01,
         -1.8049e-01,  3.5351e-01,  8.1959e-02, -4.8443e-01,  1.7036e-01,
          1.7495e-01,  2.1059e-02, -1.4918e-01, -5.4696e-03, -2.8341e-01,
         -3.7555e-01]]
fc_bias = [-0.1474,  0.0831, -0.0449, -0.1405,  0.0600,  0.0787, -0.0340, -0.0387,
        -0.1353,  0.0976]
 #signed int => unsigned int
from fxpmath import Fxp
for k in range(10):
        b = Fxp(fc_bias[k], signed=True, n_word=16, n_frac=8)
        print(b.hex())
        #print(b.val)


for i in range(10):
    for j in range(96):
        
        x = Fxp(fc_weight[i][j], signed=True, n_word=8, n_frac=4)
        #print(x)         # float value
        #print(x.bin())   # binary representation
        print(x.hex())
        #print(x.val)
        
        

#print(int_fc_weight)

#np.savetxt('fc_weight.mem', int_fc_weight, fmt='%1.2x',delimiter = " ")
#np.savetxt('fc_bias.mem', float_fc_bias, fmt='%1.2x',delimiter = " ")

Parameter containing:
tensor([-0.1474,  0.0831, -0.0449, -0.1405,  0.0600,  0.0787, -0.0340, -0.0387,
        -0.1353,  0.0976], requires_grad=True)
Parameter containing:
tensor([[-6.6225e-02,  3.2556e-02,  5.5242e-02,  6.8833e-02,  8.6716e-02,
          1.4086e-01,  8.5182e-02,  1.1414e-01, -3.7614e-02, -2.3148e-02,
         -9.5934e-02,  7.5057e-02, -4.4981e-02,  1.3153e-01, -1.3041e-01,
         -1.1367e-01, -9.4842e-03, -5.3155e-02,  8.4044e-02, -2.3950e-02,
         -1.0267e-01, -6.4267e-02,  1.5496e-01,  6.7653e-02, -3.3393e-01,
         -2.6816e-01, -1.0517e-01, -7.7368e-02, -6.3888e-02, -3.5167e-02,
         -3.7750e-02,  1.2781e-01,  3.3492e-02, -3.4174e-02,  5.9913e-02,
          1.1882e-01,  1.5604e-02,  9.1628e-02, -1.2629e-02,  2.2870e-01,
          9.1511e-02, -1.4586e-01,  8.3657e-02,  2.3657e-01,  2.4047e-01,
         -1.1409e-01, -1.7323e-01,  4.7743e-03, -2.0424e-02,  4.9587e-02,
          4.1078e-02,  2.3212e-01,  1.3312e-01,  1.3742e-01, -4.9071e-02,
          6.208

IndexError: list index out of range

In [6]:
######################### 신경망 통과 값 추출 ###########################
print(np.shape(model.conv1_out_np))
np.savetxt('out_conv1_value_1.txt', model.conv1_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv1_value_2.txt', model.conv1_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv1_value_3.txt', model.conv1_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.mp1_out_np))
np.savetxt('out_mp1_value_1.txt', model.mp1_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp1_value_2.txt', model.mp1_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp1_value_3.txt', model.mp1_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.conv3_out_np))
np.savetxt('out_conv2_value_1.txt', model.conv3_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv2_value_2.txt', model.conv3_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv2_value_3.txt', model.conv3_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.mp2_out_np))
np.savetxt('out_mp2_value_1.txt', model.mp2_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp2_value_2.txt', model.mp2_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp2_value_3.txt', model.mp2_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.fc_in_np))
np.savetxt('fc_in_value.txt', model.fc_in_np*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.fc_out_np))
np.savetxt('fc_out_value.txt', model.fc_out_np*128, fmt='%1.5d',delimiter = " ")

(16, 3, 24, 24)
(16, 3, 12, 12)
(16, 3, 8, 8)
(16, 3, 4, 4)
(16, 48)
(16, 10)


In [10]:
# 원래 입력 / 가중치 / 바이어스 출력
np.set_printoptions(suppress=True)
np.set_printoptions(precision=6)

print("Original Input")
print(model.mp1_out_np[0])
orig_input_1 = model.mp1_out_np[0][0]
orig_input_2 = model.mp1_out_np[0][1]
orig_input_3 = model.mp1_out_np[0][2]

print("Original Weight1")
print(model.conv1.weight.detach().numpy())
orig_weight = model.conv1.weight.detach().numpy()

print("Original bias1")
print(model.conv1.bias.detach().numpy())
orig_weight = model.conv1.bias.detach().numpy()

print("Original Weight2")
print(model.conv2.weight.detach().numpy())
orig_weight = model.conv2.weight.detach().numpy()

print("Original Weight3")
print(model.conv3.weight.detach().numpy())
orig_bias = model.conv3.weight.detach().numpy()

print("Original bias3")
print(model.conv3.bias.detach().numpy())
orig_weight = model.conv3.bias.detach().numpy()

orig_output_calc_1 = np.zeros((3,8,8))
orig_output_calc_2 = np.zeros((3,8,8))
orig_output_calc_3 = np.zeros((3,8,8))
orig_output_calc = np.zeros((3,8,8))

for c in range(3):
    for i in range(8):
        for j in range(8):
            orig_output_calc_1[c][i][j] += (orig_input_1[i:i+5, j:j+5] * orig_weight[c][0]).sum()
            orig_output_calc_2[c][i][j] += (orig_input_2[i:i+5, j:j+5] * orig_weight[c][1]).sum()
            orig_output_calc_3[c][i][j] += (orig_input_3[i:i+5, j:j+5] * orig_weight[c][2]).sum()         
            orig_output_calc[c][i][j] = orig_output_calc_1[c][i][j] + orig_output_calc_2[c][i][j] + orig_output_calc_3[c][i][j] + orig_bias[c]
        
print("\nBias : ")
print(orig_bias)
print("\nCalc Value :\n")
print(orig_output_calc) 
print("\nReal Value :\n")
print(model.conv2_out_np[0])

Original Input
[[[0.       0.       0.       0.       0.       0.217029 0.907689
   0.630675 0.       0.       0.       0.      ]
  [0.       0.       0.       0.       0.       1.444534 2.935081
   1.454854 0.       0.       0.       0.      ]
  [0.       0.       0.       0.       0.       2.799465 4.030156
   2.134425 0.       0.       0.       0.      ]
  [0.       0.       0.       0.       0.034406 3.356139 4.508522
   1.926755 0.       0.       0.       0.      ]
  [0.       0.       0.       0.       0.226475 3.496622 4.584289
   1.454748 0.       0.       0.       0.      ]
  [0.       0.       0.       0.       0.530351 3.7459   4.344213
   0.895292 0.       0.       0.       0.      ]
  [0.       0.       0.       0.       1.16573  4.174893 4.093958
   0.303794 0.       0.       0.       0.      ]
  [0.       0.       0.       0.       1.776142 4.344497 3.684725
   0.       0.       0.       0.       0.      ]
  [0.       0.       0.       0.       2.437443 4.511919 3.281525

IndexError: invalid index to scalar variable.

In [4]:
np.set_printoptions(precision=0)

print("Convolution Input")
print(np.shape(model.mp1_out_np))
_input = model.mp1_out_np[0] * 128
print(_input)
print('\n')

print("Convolution Weight")
print(np.shape(model.conv2.weight.detach().numpy()))
weight = model.conv2.weight.detach().numpy() * 128
print(weight)

print("Convolution Bias")
print(np.shape(model.conv2.bias.detach().numpy()))
bias = model.conv2.bias.detach().numpy() * 128
print(bias)

print("Convolution Output")
print(np.shape(model.conv2_out_np))
output = model.conv2_out_np * 128
print(output)

Convolution Input
(16, 3, 12, 12)
[[[  0.   0.   0.   0.   0.  28. 116.  81.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0. 185. 376. 186.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0. 358. 516. 273.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   4. 430. 577. 247.   0.   0.   0.   0.]
  [  0.   0.   0.   0.  29. 448. 587. 186.   0.   0.   0.   0.]
  [  0.   0.   0.   0.  68. 479. 556. 115.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 149. 534. 524.  39.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 227. 556. 472.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 312. 578. 420.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 366. 537. 300.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 371. 537. 182.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 195. 304. 105.   0.   0.   0.   0.   0.]]

 [[  0.   0.   0.   0.   0.   0.   0.   8.  40.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0. 158. 214.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0. 331. 326.   0.   0.   0.]
  [ 

AttributeError: 'NoneType' object has no attribute 'detach'

In [ ]:
print(np.shape(_input))
print(np.shape(weight))
print(np.shape(output))

_input_1 = _input[0]
_input_2 = _input[1]
_input_3 = _input[2]

output_calc_1 = np.zeros((3,8,8))
output_calc_2 = np.zeros((3,8,8))
output_calc_3 = np.zeros((3,8,8))
output_calc = np.zeros((3,8,8))

for c in range(3):
    for i in range(8):
        for j in range(8):
            output_calc_1[c][i][j] += (_input_1[i:i+5, j:j+5] * weight[c][0]).sum()
            output_calc_2[c][i][j] += (_input_2[i:i+5, j:j+5] * weight[c][1]).sum()
            output_calc_3[c][i][j] += (_input_3[i:i+5, j:j+5] * weight[c][2]).sum()
            output_calc[c][i][j] = output_calc_1[c][i][j] + output_calc_2[c][i][j] + output_calc_3[c][i][j] + bias[c] * 128
        
print("\nBias : ")
print(bias)
print("\nCalc Value :\n")
print(output_calc / 128) 
print("\nReal Value :\n")
print(output)

(3, 12, 12)
(3, 3, 5, 5)
(1, 3, 8, 8)

Bias : 
[-4. -6.  3.]

Calc Value :

[[[ -106.    90.   250.   563.   928.   929.   377.  -251.]
  [   95.   238.   569.  1235.  1621.  1524.   876.   198.]
  [   15.   207.   578.  1010.  1288.  1374.  1062.   633.]
  [ -210.    73.   534.   561.   502.   590.   786.   868.]
  [ -267.   171.   595.   195.   -97.    56.   464.   932.]
  [   15.   670.  1154.   609.    61.    40.   396.  1010.]
  [  621.  1616.  2085.  1190.   551.   492.   743.  1121.]
  [  891.  1848.  2162.  1436.   954.   903.  1092.  1141.]]

 [[  147.    75.  -150.  -377.  -206.   314.   930.  1112.]
  [ -235.  -294.  -434.  -671.  -781.  -492.    57.   635.]
  [ -132.  -258.  -328.  -362.  -310.  -373.  -284.   -93.]
  [  220.   374.   522.   452.   378.   128.    -8.  -228.]
  [  398.   921.  1310.  1186.   798.   411.   135.  -143.]
  [  -34.   426.   980.   984.   575.   167.   -74.  -113.]
  [ -749.  -695.  -445.  -217.  -185.  -292.  -411.  -273.]
  [ -589.  -831. -1024

In [ ]:
np.set_printoptions(precision=0)
print("\nCalc Value 1 :\n")
print(output_calc_1) 

print("\nCalc Value 2 :\n")
print(output_calc_2) 

print("\nCalc Value 3 :\n")
print(output_calc_3) 

print("Sum")
print(output_calc_1 + output_calc_2 + output_calc_3)


Calc Value 1 :

[[[ -79563. -115480. -119358.  -83185.  -17614.    8089.  -19178.
    -43569.]
  [ -76744.  -95861.  -57902.   21259.   71909.   58105.    3008.
    -30303.]
  [ -77871.  -72264.    -109.   69870.   77375.   31645.   -8140.
    -15770.]
  [ -86202.  -63898.    8516.   29483.      24.  -51432.  -51033.
     -8752.]
  [ -99805.  -74930.  -25465.  -50144. -102209. -119513.  -67308.
     22131.]
  [ -88203.  -55248.  -29550.  -82149. -126760. -100607.  -18583.
     77434.]
  [ -40519.    7861.   28970.  -29656.  -44251.    1708.   71418.
    120942.]
  [    207.   57557.   83441.   56935.   63897.   92173.  121985.
    118717.]]

 [[ -34046.  -46650.  -68184.  -87613.  -83399.  -52588.   -9879.
     10516.]
  [ -49032.  -62741.  -77445.  -91922.  -91984.  -72616.  -38571.
     -7807.]
  [ -50142.  -56353.  -62881.  -67248.  -65087.  -66195.  -46464.
    -24871.]
  [ -39451.  -32929.  -31157.  -33193.  -32252.  -46073.  -45074.
    -38079.]
  [ -42196.  -27861.  -15240.  -1